# Initialize

In [20]:
import csv
import pickle
import pandas as pd
import numpy as np

# Load data

In [21]:
file = open('lightcurves_1','rb')
lightcurves = pickle.load(file)

# Test code

In [26]:
# Defined parameters
#magdiff = 2 # Define the maximum difference a non-detection can have compared to the point before
timediff = 7 # In days# Define the time before the first detection where non-detections are included
#magerror = 1 # The added error in magnitude for each non-detection
noofpoint = 2 # Number of points you want before

# Listing the lists, so that they work later
R_non_mjd2_bool = []
G_non_mjd2_bool = []

R_non_mjd_additional = []
R_non_mag_additional = []
G_non_mjd_additional = []
G_non_mag_additional = []

# Loop start to work with all of them
for i in range(len(lightcurves['Data'])): # Go through each light curve
#for i in range(400):
    # Save old non-detections
    lightcurves['Data'][i]['R_non_org'] = lightcurves['Data'][i]['R_non']
    lightcurves['Data'][i]['G_non_org'] = lightcurves['Data'][i]['G_non']
    
    ## R: Red Dead Redemption Edition
    
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['R_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection(lightcurves['Data'][i]['R_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(R_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            
    else:
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection([lightcurves['Data'][i]['R_non']['mjd']]))
        
        if len(R_non_set) > 0:
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['R_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        R_non_mjd2_bool.append([float('nan')])
        R_non_mjd_additional.append([float('nan')])
        R_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag']
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd']
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err']
        
    else:
        # Start date
        R_start = lightcurves['Data'][i]['R_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        R_non_mjd2_bool.append((lightcurves['Data'][i]['R_non']['mjd'] > R_start - timediff) & (lightcurves['Data'][i]['R_non']['mjd'] < R_start)) # For those where we don't need to check for magdiff        
        
        R_non_mjd_additional.append(lightcurves['Data'][i]['R_non']['mjd'][R_non_mjd2_bool[i]])
        R_non_mag_additional.append(lightcurves['Data'][i]['R_non']['mag'][R_non_mjd2_bool[i]])

        # Get only the last two non-detections - as long as they have come within timediff
        
        R_non_mjd_additional[i] = R_non_mjd_additional[i][-noofpoint:]
        R_non_mag_additional[i] = R_non_mag_additional[i][-noofpoint:]

        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['R_mag_wn'] = np.append(R_non_mag_additional[i],lightcurves['Data'][i]['R_mag'])
        lightcurves['Data'][i]['R_mjd_wn'] = np.append(R_non_mjd_additional[i],lightcurves['Data'][i]['R_mjd'])

        # Remove NaNs
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][~np.isnan(lightcurves['Data'][i]['R_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][~np.isnan(lightcurves['Data'][i]['R_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['R_err_wn'] = np.append(np.repeat(-1,len(R_non_mag_additional[i])),lightcurves['Data'][i]['R_err'])
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][~np.isnan(lightcurves['Data'][i]['R_err_wn'])] # remove NaN again
    
    ## G: Green New Deal Edition
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['G_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection(lightcurves['Data'][i]['G_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(G_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            
    else:
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection([lightcurves['Data'][i]['G_non']['mjd']]))
        
        if len(G_non_set) > 0:
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['G_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        G_non_mjd2_bool.append([float('nan')])
        G_non_mjd_additional.append([float('nan')])
        G_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag']
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd']
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err']
        
    else:
        # Start date
        G_start = lightcurves['Data'][i]['G_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        G_non_mjd2_bool.append((lightcurves['Data'][i]['G_non']['mjd'] > G_start - timediff) & (lightcurves['Data'][i]['G_non']['mjd'] < G_start)) # For those where we don't need to check for magdiff

        G_non_mjd_additional.append(lightcurves['Data'][i]['G_non']['mjd'][G_non_mjd2_bool[i]])
        G_non_mag_additional.append(lightcurves['Data'][i]['G_non']['mag'][G_non_mjd2_bool[i]])

        # Get only the last two non-detections - as long as they have come within timediff
        
        G_non_mjd_additional[i] = G_non_mjd_additional[i][-noofpoint:]
        G_non_mag_additional[i] = G_non_mag_additional[i][-noofpoint:]


        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['G_mag_wn'] = np.append(G_non_mag_additional[i],lightcurves['Data'][i]['G_mag'])
        lightcurves['Data'][i]['G_mjd_wn'] = np.append(G_non_mjd_additional[i],lightcurves['Data'][i]['G_mjd'])

        # Remove NaNs
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][~np.isnan(lightcurves['Data'][i]['G_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][~np.isnan(lightcurves['Data'][i]['G_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['G_err_wn'] = np.append(np.repeat(-1,len(G_non_mag_additional[i])),lightcurves['Data'][i]['G_err'])
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][~np.isnan(lightcurves['Data'][i]['G_err_wn'])] # remove NaN again
        

/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


# Function

In [ ]:
def dettwopredet(lightcurves,timediff,noofpoint)
    # Listing the lists, so that they work later
    R_non_mjd2_bool = []
    G_non_mjd2_bool = []

    R_non_mjd_additional = []
    R_non_mag_additional = []
    G_non_mjd_additional = []
    G_non_mag_additional = []

    # Loop start to work with all of them
    for i in range(len(lightcurves['Data'])): # Go through each light curve
    #for i in range(400):
        # Save old non-detections
        lightcurves['Data'][i]['R_non_org'] = lightcurves['Data'][i]['R_non']
        lightcurves['Data'][i]['G_non_org'] = lightcurves['Data'][i]['G_non']

        ## R: Red Dead Redemption Edition

        # Remove all non-detections that have occured at the same time as a real detection
        if type(lightcurves['Data'][i]['R_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
            R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection(lightcurves['Data'][i]['R_non']['mjd'])) # Then find the intersection between non-detections and detections

            if len(R_non_set) > 0: # if it is non-empty
                lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
                lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]

        else:
            R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection([lightcurves['Data'][i]['R_non']['mjd']]))

            if len(R_non_set) > 0:
                lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
                lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]

        # Check if code has put in a -1:

        if type(lightcurves['Data'][i]['R_non']['mjd']) == int:
            # Give all of them an empty array so that it works
            R_non_mjd2_bool.append([float('nan')])
            R_non_mjd_additional.append([float('nan')])
            R_non_mag_additional.append([float('nan')])

            # Put in the actual lightcurves
            lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag']
            lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd']
            lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err']

        else:
            # Start date
            R_start = lightcurves['Data'][i]['R_mjd'][0] # Find the start date

            # Find the non-detections that lies timediff before we start
            R_non_mjd2_bool.append((lightcurves['Data'][i]['R_non']['mjd'] > R_start - timediff) & (lightcurves['Data'][i]['R_non']['mjd'] < R_start)) # For those where we don't need to check for magdiff        

            R_non_mjd_additional.append(lightcurves['Data'][i]['R_non']['mjd'][R_non_mjd2_bool[i]])
            R_non_mag_additional.append(lightcurves['Data'][i]['R_non']['mag'][R_non_mjd2_bool[i]])

            # Get only the last two non-detections - as long as they have come within timediff

            R_non_mjd_additional[i] = R_non_mjd_additional[i][-noofpoint:]
            R_non_mag_additional[i] = R_non_mag_additional[i][-noofpoint:]

            # The new data vectors are added to the existing vectors

            lightcurves['Data'][i]['R_mag_wn'] = np.append(R_non_mag_additional[i],lightcurves['Data'][i]['R_mag'])
            lightcurves['Data'][i]['R_mjd_wn'] = np.append(R_non_mjd_additional[i],lightcurves['Data'][i]['R_mjd'])

            # Remove NaNs
            lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][~np.isnan(lightcurves['Data'][i]['R_mag_wn'])] # Removes NaNs
            lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][~np.isnan(lightcurves['Data'][i]['R_mjd_wn'])]

            # A new error list is added as well, with magerror being put in
            lightcurves['Data'][i]['R_err_wn'] = np.append(np.repeat(-1,len(R_non_mag_additional[i])),lightcurves['Data'][i]['R_err'])
            lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][~np.isnan(lightcurves['Data'][i]['R_err_wn'])] # remove NaN again

        ## G: Green New Deal Edition
        # Remove all non-detections that have occured at the same time as a real detection
        if type(lightcurves['Data'][i]['G_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
            G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection(lightcurves['Data'][i]['G_non']['mjd'])) # Then find the intersection between non-detections and detections

            if len(G_non_set) > 0: # if it is non-empty
                lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
                lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]

        else:
            G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection([lightcurves['Data'][i]['G_non']['mjd']]))

            if len(G_non_set) > 0:
                lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
                lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]

        # Check if code has put in a -1:

        if type(lightcurves['Data'][i]['G_non']['mjd']) == int:
            # Give all of them an empty array so that it works
            G_non_mjd2_bool.append([float('nan')])
            G_non_mjd_additional.append([float('nan')])
            G_non_mag_additional.append([float('nan')])

            # Put in the actual lightcurves
            lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag']
            lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd']
            lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err']

        else:
            # Start date
            G_start = lightcurves['Data'][i]['G_mjd'][0] # Find the start date

            # Find the non-detections that lies timediff before we start
            G_non_mjd2_bool.append((lightcurves['Data'][i]['G_non']['mjd'] > G_start - timediff) & (lightcurves['Data'][i]['G_non']['mjd'] < G_start)) # For those where we don't need to check for magdiff

            G_non_mjd_additional.append(lightcurves['Data'][i]['G_non']['mjd'][G_non_mjd2_bool[i]])
            G_non_mag_additional.append(lightcurves['Data'][i]['G_non']['mag'][G_non_mjd2_bool[i]])

            # Get only the last two non-detections - as long as they have come within timediff

            G_non_mjd_additional[i] = G_non_mjd_additional[i][-noofpoint:]
            G_non_mag_additional[i] = G_non_mag_additional[i][-noofpoint:]


            # The new data vectors are added to the existing vectors

            lightcurves['Data'][i]['G_mag_wn'] = np.append(G_non_mag_additional[i],lightcurves['Data'][i]['G_mag'])
            lightcurves['Data'][i]['G_mjd_wn'] = np.append(G_non_mjd_additional[i],lightcurves['Data'][i]['G_mjd'])

            # Remove NaNs
            lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][~np.isnan(lightcurves['Data'][i]['G_mag_wn'])] # Removes NaNs
            lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][~np.isnan(lightcurves['Data'][i]['G_mjd_wn'])]

            # A new error list is added as well, with magerror being put in
            lightcurves['Data'][i]['G_err_wn'] = np.append(np.repeat(-1,len(G_non_mag_additional[i])),lightcurves['Data'][i]['G_err'])
            lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][~np.isnan(lightcurves['Data'][i]['G_err_wn'])] # remove NaN again
